# GridSearchCV - 교차 검증과 최적 하이퍼 파라미터 튜닝을 한 번에 

사이킷런은 GridSearchCV API를 이용해 Classifier나 Regressor 같은 알고리즘에 사용되는 하이퍼 파아미터를 순차적으로 입력하면서 편리하게 최적의 파라미터를 도출할 수 있는 방안을 제공합니다.

GridSearchCV는 교차 검증을 기반으로 하이퍼 파라미터의 최적 값을 찾게 해줍니다.

즉 , 데이터 세트를 cross-validation을 위한 학습/테스트 세트로 자동 분할한 뒤에 하이퍼 파라미터 그리드에 기술된 모든 파라미터를 순차적으로 적용해 최적의 파라미터를 찾을 수 있게 해줍니다.

GridSearchCV는 사용자가 튜닝하고자 하는 여러 종류의 하이퍼 파라미터를 다양하게 테스트하면서 최적의 파라미터를 편리하게 찾게 해주지만 동시에 순차적으로 파라미터를 테스트하므로 수행시간이 상대적으로 오래 걸리는 것에 유념해야 합니다.

GridSearchCV 클래스의 생성자로 들어가는 주요 파라미터는 다음과 같습니다.
- estimator: classifier, regressor, pipeline 등
- param_grid: key+리스트 값을 가지는 딕셔너리가 주어집니다. estimator의 튜닝을 위해 파라미터명과 사용될 여러 파라미터 값을 지정
- scoring: 예측 성능을 측정할 평가 방법을 지정합니다.
- cv: 교차 검증을 위해 분할되는 학습/테스트 세트의 개수를 지정합니다.
- refit: 디폴트가 True이며 True로 생성 시 가장 최적의 하이퍼 파라미터를 찾은 뒤 입력된 estimator 객체를 재학습시킵니다.

In [23]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

### 데이터를 로딩하고 학습 데이터와 테스트 데이터 분리 

In [5]:
iris_data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target,
                                                   test_size=0.2, random_state=121)

In [6]:
dtree = DecisionTreeClassifier()

### 파라미터를 딕셔너리 형태로 설정 

In [15]:
parameters = {'max_depth': [1, 2, 3], 'min_samples_split': [2, 3]}

In [16]:
import pandas as pd

### param_grid의 하이퍼 파라미터를 3개의 train, test set fold로 나누어 테스트 수행 설정 

In [17]:
### refit=True가 default임. True면 가장 좋은 파라미터 설정으로 재학습시킴.

In [18]:
grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True)

### 붓꽃 학습 데이터로 param_grid의 하이퍼 파라미터를 순차적으로 학습/평가 

In [19]:
grid_dtree.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [1, 2, 3], 'min_samples_split': [2, 3]})

### GridSearchCV 결과를 추출해 DataFrame으로 변환 

In [20]:
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', 'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


위의 결과에서 총 6개의 결과를 볼 수 있으며, 이는 하이퍼 파라미터 max_depth와 min_samples_split을 순차적으로 6번 변경하면서 학습 및 평가를 수행했음을 나타냅니다.

'rank_test_score'칼럼 값 1의 의미는 예측 성능이 1위라는 의미입니다.

In [21]:
print('GridSearchCV 최적 파라미터:', grid_dtree.best_params_)
print('GridSearchCV 최고 정확도:{0:.4f}'.format(grid_dtree.best_score_))

GridSearchCV 최적 파라미터: {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도:0.9750


max_depth가 3, min_samples_split 2일 때 검증용 폴드 세트에서 평균 최고 정확도가 96.67%로 측정됐습니다.

### GridSearchCV의 refit으로 이미 학습된 estimator 반환 

In [22]:
estimator = grid_dtree.best_estimator_

### GridSearchCV의 best_estimator_는 이미 최적 학습이 됐으므로 별도 학습 필요 없음 

In [24]:
pred = estimator.predict(X_test)
print("테스트 데이터 세트 정확도: {0:.4f}".format(accuracy_score(y_test,pred)))

테스트 데이터 세트 정확도: 0.9667


별도의 테스트 데이터 세트로 정확도를 측정한 결과 약 96.67%의 결과가 도출됐습니다.

일반적으로 학습 데이터를 GridSearchCV를 이용해 최적 하이퍼 파라미터 튜닝을 수행한 뒤에 별도의 테스트 세트에서 이를 평가하는 것이 일반적인 머신러닝 모델 적용 방법입니다.